In [70]:
import pickle
import pprint
import matplotlib.pyplot as plt
import numpy as np
import os

sys.path.append("..")
from demo_2_awac import och_2_awac

BASE_PATH = '/usr/local/google/home/bkinman/proj/rpl_reset_free/20201020_slider_open_close/'

OPEN_SLIDER_FILE = os.path.join(BASE_PATH, 'open_slider_20x_0p15_noise.pkl')
CLOSE_SLIDER_FILE = os.path.join(BASE_PATH, 'close_slider_20x_0p15_noise.pkl')
GOTO_SLIDER_FILE = os.path.join(BASE_PATH, 'goto_near_slider_20x_0p15_noise.pkl')
GOTO_HINGE_FILE = os.path.join(BASE_PATH, 'goto_near_hinge_20x_0p15_noise.pkl')

def awac_dict_from_pkl(pkl_path):
    """ Converts demonstration vector pickle to awac dict. """
    full_awac_dict = None
    data = pickle.load(open(pkl_path,'rb'))
    awac_formatted_list = och_2_awac(data)
    return awac_formatted_list

def bc_make_goal(awac_dict):
    """Set the goal vector for every step in a each rollout to be the final state of the trajectory."""
    for demo in awac_dict:
        last_obs = demo['observations'][-1][:25]
        last_obs = np.ones(last_obs.shape)
        demo['observations'][:,25:] = last_obs


In [71]:
# Make behavior cloning demo.
slider_demos_awac = awac_dict_from_pkl(GOTO_SLIDER_FILE)
bc_make_goal(slider_demos_awac)
hinge_demos_awac = awac_dict_from_pkl(GOTO_HINGE_FILE)
bc_make_goal(hinge_demos_awac)

combined_demos_awac = slider_demos_awac + hinge_demos_awac

lens = sum(len(a['observations']) for a in combined_demos_awac)
print(f'bc_num_pretrain_steps should be {(int(lens/1000)+1)*1500} steps')
print(f'q_num_pretrain2_steps should be {(int(lens/1000)+1)*3000} steps')

pickle.dump(combined_demos_awac, open(os.path.join(BASE_PATH, 'goto_bc_combined.pkl') , 'wb'))

bc_num_pretrain_steps should be 6000 steps
q_num_pretrain2_steps should be 12000 steps


In [ ]:
# Make behavior cloning demo for just 